# 03 - Data Transformations with Spark SQL

## Overview
This notebook applies data transformations to the streaming transaction data using pure Spark SQL.

## Transformation Goals
1. Cast string timestamps to proper timestamp type
2. Filter out invalid or incomplete records
3. Handle null values appropriately
4. Create derived columns for analytics
5. Standardize data formats

## Architecture Pattern
All business logic is maintained in external `.sql` files under the `sql/` directory. This approach:
- Separates concerns (logic vs orchestration)
- Enables SQL-first development
- Facilitates version control and testing
- Allows non-Python developers to contribute

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pathlib import Path
import os

## Initialize or Retrieve Spark Session

If continuing from notebook 02, retrieve the existing session. Otherwise, create a new one.

In [ ]:
# Get existing Spark session or create new one
try:
    spark = SparkSession.getActiveSession()
    if spark is None:
        raise Exception("No active session")
    print("Using existing Spark session")
except:
    spark = SparkSession.builder \
        .appName("TransactionStreamingETL") \
        .master("local[*]") \
        .config("spark.sql.streaming.schemaInference", "false") \
        .config("spark.sql.shuffle.partitions", "4") \
        .getOrCreate()
    print("Created new Spark session")

spark.sparkContext.setLogLevel("WARN")
print(f"Spark Version: {spark.version}")

## Configure Paths

In [ ]:
# Configure paths
BASE_DIR = Path(os.path.abspath('')).parent
SQL_DIR = BASE_DIR / 'sql'
INPUT_DIR = str(BASE_DIR / 'data' / 'input')

print(f"SQL Directory: {SQL_DIR}")
print(f"Input Directory: {INPUT_DIR}")

## Set Up Streaming Source

If not already set up, recreate the streaming source and temporary view.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Define schema (same as notebook 02)
transaction_schema = StructType([
    StructField("transaction_id", StringType(), False),
    StructField("user_id", StringType(), False),
    StructField("product_id", StringType(), False),
    StructField("product_category", StringType(), True),
    StructField("amount", DoubleType(), False),
    StructField("quantity", IntegerType(), False),
    StructField("payment_method", StringType(), True),
    StructField("status", StringType(), False),
    StructField("event_time", StringType(), False),
    StructField("country_code", StringType(), True),
    StructField("discount_percent", DoubleType(), True),
    StructField("customer_segment", StringType(), True)
])

# Check if view already exists, otherwise create it
existing_views = [table.name for table in spark.catalog.listTables() if table.isTemporary]

if 'raw_transactions' not in existing_views:
    print("Creating streaming source...")
    raw_stream = spark.readStream \
        .format("csv") \
        .schema(transaction_schema) \
        .option("header", "true") \
        .option("maxFilesPerTrigger", 1) \
        .load(INPUT_DIR)
    
    raw_stream.createOrReplaceTempView("raw_transactions")
    print("Streaming source created and registered as 'raw_transactions'")
else:
    print("Using existing 'raw_transactions' view")

## Load SQL Transformation Query

Load the transformation logic from the external SQL file. This file contains all the business rules for cleaning and enriching the data.

In [ ]:
# Load SQL from external file
sql_file_path = SQL_DIR / 'transformations.sql'

with open(sql_file_path, 'r') as f:
    transformation_sql = f.read()

print(f"Loaded SQL from: {sql_file_path}")
print(f"\nSQL Query ({len(transformation_sql)} characters):")
print("=" * 80)
print(transformation_sql)
print("=" * 80)

## Execute Transformation

Apply the SQL transformation to create a cleaned and enriched streaming DataFrame.

In [ ]:
# Execute transformation SQL
transformed_stream = spark.sql(transformation_sql)

print("Transformation applied successfully!")
print(f"Is Streaming: {transformed_stream.isStreaming}")
print(f"\nTransformed Schema:")
transformed_stream.printSchema()

## Register Transformed View

Register the transformed data as a new temporary view for downstream processing.

In [ ]:
# Register transformed stream as temporary view
transformed_stream.createOrReplaceTempView("transformed_transactions")

print("Registered as 'transformed_transactions' view")
print("This view is ready for aggregations and analytics")

## Validate Transformations

Execute a test query to verify the transformations are working correctly.

In [ ]:
# Test query to validate transformations
validation_query = spark.sql("""
    SELECT 
        transaction_id,
        user_id,
        product_category,
        amount,
        discounted_amount,
        revenue,
        is_high_value,
        status,
        event_timestamp,
        event_date,
        event_hour
    FROM transformed_transactions
""")

print("Validation query created")
print(f"Is Streaming: {validation_query.isStreaming}")

## Run Test Output

Display sample transformed records to console for verification.

In [ ]:
# Write to console for validation
validation_stream = validation_query.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 10) \
    .trigger(processingTime='5 seconds') \
    .start()

print("Validation query started...")
print(f"Query ID: {validation_stream.id}")

In [ ]:
# Let it run briefly
import time
time.sleep(20)

# Stop validation query
validation_stream.stop()
print("Validation query stopped.")

## Transformation Summary

Review what transformations were applied by querying the statistics.

In [ ]:
# Check column list
print("Transformed Columns:")
for col_name in transformed_stream.columns:
    print(f"  - {col_name}")

## Summary

This notebook successfully:

1. Loaded SQL transformation logic from external file
2. Applied data cleaning and validation rules
3. Cast timestamp strings to proper timestamp types
4. Created derived columns (revenue, flags, date parts)
5. Handled null values with COALESCE
6. Registered transformed data as SQL view

**Key Transformations Applied:**
- Timestamp parsing and date extraction
- Revenue calculation with discount logic
- High-value transaction flagging
- Status categorization
- Invalid record filtering

**Benefits of SQL-Based Transformations:**
- Declarative and readable
- Catalyst optimizer handles execution
- Version controlled separately
- Easy to test and modify

**Next Steps:**
- Proceed to notebook 04 for aggregations
- Calculate windowed metrics and KPIs
- Perform stateful operations